# Breast Cancer with mutation "survival model"

_Survival Analysis_ is a set of statistical tools, which addresses questions such as ‘*how long would it be, before a particular event occurs*’; in other words we can also call it as a ‘*time to event*’ analysis. Of particulat interst to this estimation is "*what is the likelihood that a patient will survive, after being diagnosed of breast cancer with a small chance of the cancer mutating?*" 

## Problem to solve
###  Calculate the estimation accuracy when the study sample size is 1500. 

In [7]:
brca_sample_size = 1500

Using published BReast CAncer (BRCA) germline mutation rates (5.6%), approximately 84 early patients were positive for BRCA mutations. Assuming 76 patients survived for 5 years (90%), regardless of the sample size, at least 10 years of follow-up is required to estimate the half-width of the 95% confidence interval. Therefore, the survival analysis will calculate the survival rate of 2, 3, 4, 5, 6 years. The 5-year survival rate is about 90%, which can be estimated as 95% CI, ie +/- 10%, and 85% to 95% is acceptable.

In [8]:
p1=.90
p_diff=.05
alpha=.05

* Group **BRCA_Mutation** - those who *have* mutated, given they have BRCA
* Group **BRCA** - those who *have NOT* mutated, given they have BRCA
* Group **Neither** - those from the sample that do not have BRCA

[First calculate the CI](http://napitupulu-jon.appspot.com/posts/confidence-interval-coursera-statistics.html) - to ensure that we don't by any chance miss the population parameter and have a good chance of capturing the BRCA patients with mutation (i.e. BRCA_Mutation) a 95% CI would require a sample size of: 

In [9]:
import math
import numpy as np
from scipy import stats
import pandas as pd
import seaborn as sns

def z_calc(p1, p2, n1, n2):
    p_star = (p1*n1 + p2*n2) / (n1 + n2)
    return (p2 - p1) / math.sqrt(p_star*(1 - p_star)*((1.0 / n1) + (1.0 / n2)))

def sample_required(p1, p_diff, alpha):
    if p_diff <= 0:
        raise ValueError("p_diff must be > 0")
    n = 1
    while True:
        z = z_calc(p1, p1+p_diff, n1=n, n2=n)
        p = 1 - stats.norm.cdf(z)
        if p < alpha:
            break
        n += 1
    return n

print(str(sample_required(p1, p_diff, alpha)))

151


Using published BReast CAncer (BRCA) germline mutation rates (5.6%), approximately 84 early patients were positive for BRCA mutations. Assuming 76 patients survived for 5 years (90%), regardless of the sample size, at least 10 years of follow-up is required to estimate the half-width of the 95% confidence interval. Therefore, the survival analysis will calculate the survival rate of 2, 3, 4, 5, 6 years. The 5-year survival rate is about 90%, which can be estimated as 95% CI, ie +/- 10%, and 85% to 95% is acceptable.

* Group **BRCA_Mutation** - those who *have* mutated, given they have BRCA
* Group **BRCA** - those who *have NOT* mutated, given they have BRCA
* Group **Neither** - those from the sample that do not have BRCA

[First calculate the CI](http://napitupulu-jon.appspot.com/posts/confidence-interval-coursera-statistics.html) - to ensure that we don't by any chance miss the population parameter and have a good chance of capturing the BRCA patients with mutation (i.e. BRCA_Mutation) a 95% CI would require a sample size of: 

In [13]:
## sample size
sample_size = sample_required(p1, p_diff, alpha)
## mutation rate of the poisitve for BRCA mutation
mutation_rate = 0.056
## approx. 84 patients of the 1500 were positive for BRCA mutation
print ("number of positive cases for BRCA mutation, with a mutation rate of {} form the sample size of {}, is ".format(mutation_rate, sample_size,sample_size*mutation_rate))
early_patients_positive = 84
## Assuming 76 patients survived for 5 years (90%)
survival_time_yrs = 5
num_patients_survive_5yrs = 76
print ("number of postive cases that survived for 5 years = {}".format(num_patients_survive_5yrs/early_patients_positive))
## < 10 years of follow-up is required to estimate the half-width of the 95% confidence interval
half_width_95pcent_conf = 76

number of positive cases for BRCA mutation, with a mutation rate of 0.056 form the sample size of 151, is 
number of postive cases that survived for 5 years = 0.9047619047619048


### Preliminaries

Consider a random variable that has a **Probability Density Function** f(t), and a **Cumulative Distribution Function** F(t): As per the definition of CDF from a given PDF, we can define cdf as F(t) = P (T< t); here, F(t) gives us the probability that the event has occurred by duration t. In simple words, F(t) gives us the proportion of population who will die (or survive) with a time value < t.

**Survival Function**: S(t) = 1 - F(t) = P(T ≥t); S(t) gives us the probability that the event has not occurred by the time t. In simple words, S(t) gives us the proportion of population that will survive after a period of time > t.

**Hazard Function**: h(t) Along with the survival function, we are also interested in the rate at which event is taking place, out of the surviving population at any given time t. In medical terms, we can define it as “out of the people who survived at time t, what is the rate of dying of those people”.

h(t) = [( S(t) -S(t + dt) )/dt] / S(t) ... the rate of tying at time t

limit dt → 0

[1] Survival Analysis: Intuition & Implementation in Python: https://towardsdatascience.com/survival-analysis-intuition-implementation-in-python-504fde4fcf8e  

In [14]:
## The rate of dying is 10% at the fith year
##
h = 0.1
print ('Instantaneous rate of dying T greater than 5 years = ', early_patients_positive * h)

## Therefore, the survival analysis will calculate the 'survival rate' for each event of interest at time T > 0
## Radon variable 'T' denotes the time from diagnosis to the death of a randomly selected patient
## Pr(T > 5 | ) = 0.1 of the diagnosed will die.
##
t = [0, 1, 2, 3, 4, 5, 6]
print('Observations are made at year t =', str(t)[1:-1])

## estimated as 95% confidence interval, ie +/- 10%, and from 85% to 95%
##
desired_confidence_interval = [.85, .95]

Instantaneous rate of dying T greater than 5 years =  8.4
Observations are made at year t = 0, 1, 2, 3, 4, 5, 6


## Methods for achieving "estimation accuracy"

When you know the actual functional form of the hazard function, the **fully parametric survival model** is far more efficient than the **Cox model**. Statistical efficiency is like power. A good way to think of it is the width of the confidence interval for your final estimate of the log-hazard ratios: a tight CI is the result of an efficient analysis (assuming you have an unbiased estimator [2]. The advantages of _Cox Proportional Hazards regression over fully parametric ones, without assumption on the distribution of the survival time_ -- If you know the parametric distribution that your data follows then using a maximum likelihood approach and the distribution makes sense. The real advantage of Cox Proportional Hazards regression is that you can still fit survival models without knowing (or assuming) the distribution [3].

[2] In survival analysis, when should we use fully parametric models over semi-parametric ones? https://stats.stackexchange.com/questions/399544/in-survival-analysis-when-should-we-use-fully-parametric-models-over-semi-param

[3] In survival analysis, why do we use semi-parametric models (Cox proportional hazards) instead of fully parametric models? https://stats.stackexchange.com/questions/64739/in-survival-analysis-why-do-we-use-semi-parametric-models-cox-proportional-haz/64743#64743 

[4] The restricted cubic spline as baseline hazard in the proportional hazards model with step function time‐dependent covariables: https://onlinelibrary.wiley.com/doi/pdf/10.1002/sim.4780141906

### Kaplan-Meier estimator
This is a non-parametric method called the Kaplan-Meier estimator and comes with the ["lifelines"](https://lifelines.readthedocs.io/en/latest/Survival%20Analysis%20intro.html) python library. In the KM estimation,**n(i)** is deﬁned as the population at risk at time just prior to time **t(i)**; and **d(i)** is defined as number of events occurred at time *t(i)*. This, will become more clear with the example below. The **inclusion of censored data to calculate the estimates**, makes the *KM survival analysis* very powerful, and it stands out as compared to many other statistical techniques.

Mathematically, for any time t ∈ [t1, t2), we have S(t) = P(survive in [0, t1)) × P(survive in [t1, t] | survive in [0, t1))

In [16]:
n=[0 for i in range(len(t)-1)]
n[0] = early_patients_positive
n[4] = num_patients_survive_5yrs
print('Initializing numner of cases at risk n =', str(n)[1:-1])
s=[1 for i in range(len(t)-1)]
print('Number of positive cases at risk; n(t=0) =',str(n[0]))
print('Number of positive cases at risk; n(t=4) =',str(n[4]))
print('Initializing survival rates for each year; s=', str(s)[1:-1])
##
## Number of mutations (events) that occured at time t=0 is 0, implies d(0)=0
## The probability that a positive case can mutate after 1 year is 5.6%
d = [mutation_rate for i in range(len(t)-1)]
d[0]=0
d[4]=n[4]*mutation_rate
print('Initializing mutation rates for each year; d=', str(d)[1:-1])
##
## Python code to create the above Kaplan Meier curve
from lifelines import KaplanMeierFitter



Initializing numner of cases at risk n = 84, 0, 0, 0, 76, 0
Number of positive cases at risk; n(t=0) = 84
Number of positive cases at risk; n(t=4) = 76
Initializing survival rates for each year; s= 1, 1, 1, 1, 1, 1
Initializing mutation rates for each year; d= 0, 0.056, 0.056, 0.056, 4.256, 0.056


### Cox propotional hazard model
